In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================
# 설정: 파일 경로 지정
# ============================
WORD_FILE  = '../community/monthly_word_alt.csv'               # 예: 'monthly_word_alt.csv'
# WORD_FILE  = './community/monthly_word_Chart.csv'               # 예: 'monthly_word_alt.csv'
# WORD_FILE  = './community/monthly_word_electronicmoney.csv'               # 예: 'monthly_word_alt.csv'
CHART_FILE = '../chart/BINANCE_BTCUSDT_daily_UTC.csv'

# ============================
# 1) 월별 단어 빈도 데이터 로드
# ============================
# 파일이 “콤마(,) 구분된 CSV”이므로 sep=',' 사용
df_words = pd.read_csv(
    WORD_FILE,
    sep=',',
    encoding='utf-8-sig'
)
# 컬럼: ['month', 'count', 'rankw', 'word', 'frequency']
print("읽은 df_words 컬럼명:", df_words.columns.tolist())

# ============================
# 2) df_chart(일별 BTCUSDT 차트) 데이터 로드
# ============================
df_chart = pd.read_csv(
    CHART_FILE,
    parse_dates=['date']
)

# ============================
# 3) 데이터 전처리: 'year_month' 생성 & 월별 등락률 계산
# ============================
# 3-1) df_words: 'month' 열이 이미 'YYYY-MM' 형식이므로 바로 활용
df_words['year_month'] = df_words['month']  # 예: "2020-01", "2020-02", …

# 3-2) df_chart: 'date' → 'year_month'(period) → 시가/종가 집계 → 등락률 계산
df_chart['year_month'] = df_chart['date'].dt.to_period('M')
monthly_prices = (
    df_chart
    .groupby('year_month')['close']
    .agg(first='first', last='last')
    .reset_index()
)
monthly_prices['pct_change'] = (
    monthly_prices['last'] - monthly_prices['first']
) / monthly_prices['first']
monthly_prices['year_month'] = monthly_prices['year_month'].dt.strftime('%Y-%m')

# 3-3) 등락률 기준으로 카테고리 분류: down(≤-10%), neutral(–10%~+10%), up(≥+10%)
monthly_prices['category'] = pd.cut(
    monthly_prices['pct_change'],
    bins=[-np.inf, -0.10, 0.10, np.inf],
    labels=['down', 'neutral', 'up']
)

print("\n월별 등락률 예시:")
print(monthly_prices[['year_month','first','last','pct_change','category']].head())

# ============================
# 4) 병합 및 상위 단어 추출
# ============================
df_merged = pd.merge(
    df_words,
    monthly_prices[['year_month','category','pct_change']],
    on='year_month',
    how='inner'
)

# 4-1) 카테고리별 단어 빈도 합계 → 상위 5개
category_word_freq = (
    df_merged
    .groupby(['category','word'])['frequency']
    .sum()
    .reset_index()
)
top_words_by_category = (
    category_word_freq
    .sort_values(['category','frequency'], ascending=[True, False])
    .groupby('category')
    .head(5)
    .reset_index(drop=True)
)

print("\n=== 카테고리별 상위 5개 단어 (빈도 합계 기준) ===")
print(top_words_by_category.to_string(index=False))

# ============================
# 5) 단어 빈도와 등락률 상관계수 계산
# ============================
pivot_words = df_merged.pivot_table(
    index='year_month',
    columns='word',
    values='frequency',
    aggfunc='sum',
    fill_value=0
).reset_index()

corr_df = pd.merge(
    pivot_words,
    monthly_prices[['year_month','pct_change']],
    on='year_month',
    how='inner'
)

word_columns = corr_df.columns.drop(['year_month','pct_change'])
corr_series = corr_df[word_columns].corrwith(corr_df['pct_change']).sort_values(ascending=False)

top_positive_corr = corr_series.head(5).reset_index()
top_positive_corr.columns = ['단어','상관계수']
top_negative_corr = corr_series.tail(5).reset_index()
top_negative_corr.columns = ['단어','상관계수']

print("\n=== pct_change와 양(+) 상관 상위 5개 단어 ===")
print(top_positive_corr.round(4).to_string(index=False))

print("\n=== pct_change와 음(-) 상관 상위 5개 단어 ===")
print(top_negative_corr.round(4).to_string(index=False))

# ============================
# 6) CNN 모델 학습을 위한 데이터 준비
# ============================
X = corr_df[word_columns].values                   # (월 수, 단어 수)
X = X.reshape((X.shape[0], X.shape[1], 1))          # Conv1D 입력 형태: (samples, timesteps, channels)

le = LabelEncoder()
y = le.fit_transform(monthly_prices['category'])   # down→0, neutral→1, up→2

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ============================
# 7) 1D CNN 모델 정의 및 학습
# ============================
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1],1)),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

es = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

print("\n=== CNN 모델 학습 시작 ===")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=8,
    validation_split=0.1,
    callbacks=[es],
    verbose=2
)

# ============================
# 8) 모델 평가 및 결과 출력
# ============================
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n=== 테스트 세트 평가 ===\nLoss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# ============================
# 9) 모델링 기법 설명
# ============================
print("\n=== 모델링 기법 설명 ===")
print(
    "이 스크립트에서는 '월별 단어 빈도 벡터'를 1차원 시계열처럼 보고,\n"
    "Conv1D(1D Convolutional Neural Network) 레이어를 통해 특징을 추출한 뒤\n"
    "GlobalMaxPooling1D → Dense → Softmax 구조로 'down', 'neutral', 'up' 세 클래스를 예측합니다.\n"
    "즉, 단어 빈도 패턴을 CNN이 학습하여 BTC 월별 등락 카테고리를 분류하는 모델링 방식입니다."
)

읽은 df_words 컬럼명: ['month', 'count', 'rank', 'word', 'frequency']

월별 등락률 예시:
  year_month    first     last  pct_change category
0    2020-01  9352.89  7200.85   -0.230094     down
1    2020-02  8523.61  9384.61    0.101014       up
2    2020-03  6410.44  8531.88    0.330935       up
3    2020-04  8620.00  6642.92   -0.229360     down
4    2020-05  9448.27  8826.96   -0.065759  neutral

=== 카테고리별 상위 5개 단어 (빈도 합계 기준) ===
category word  frequency
    down   코인       2519
    down   비트       2168
    down   버그       1264
    down   사람       1221
    down   따리        787
 neutral   코인       1411
 neutral   비트       1168
 neutral   사람        783
 neutral   정신        705
 neutral   존귀        438
      up   코인       1487
      up   비트       1292
      up  하나님       1211
      up   사람        855
      up   정신        513

=== pct_change와 양(+) 상관 상위 5개 단어 ===
 단어   상관계수
 팬티 0.4058
 누나 0.3510
 승현 0.3510
 치기 0.3510
박치기 0.3510

=== pct_change와 음(-) 상관 상위 5개 단어 ===
  단어    상관계수
  알트 -0.2647
  리퍼 -0.

C:\Users\kmj\AppData\Local\Temp\ipykernel_16720\3626450370.py:79: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['category','word'])['frequency']
C:\Users\kmj\AppData\Local\Temp\ipykernel_16720\3626450370.py:86: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('category')
c:\Users\kmj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activ


=== CNN 모델 학습 시작 ===
Epoch 1/20
6/6 - 2s - 265ms/step - accuracy: 0.2667 - loss: 4.8831 - val_accuracy: 0.1667 - val_loss: 4.5767
Epoch 2/20
6/6 - 0s - 19ms/step - accuracy: 0.2667 - loss: 2.0589 - val_accuracy: 0.5000 - val_loss: 1.5435
Epoch 3/20
6/6 - 0s - 18ms/step - accuracy: 0.2444 - loss: 1.7782 - val_accuracy: 0.1667 - val_loss: 1.6613
Epoch 4/20
6/6 - 0s - 22ms/step - accuracy: 0.2222 - loss: 1.7759 - val_accuracy: 0.5000 - val_loss: 0.9612
Epoch 5/20
6/6 - 0s - 19ms/step - accuracy: 0.3111 - loss: 1.3591 - val_accuracy: 0.1667 - val_loss: 3.6327
Epoch 6/20
6/6 - 0s - 19ms/step - accuracy: 0.3778 - loss: 1.7670 - val_accuracy: 0.3333 - val_loss: 1.4312
Epoch 7/20
6/6 - 0s - 21ms/step - accuracy: 0.3111 - loss: 1.2275 - val_accuracy: 0.5000 - val_loss: 1.0112

=== 테스트 세트 평가 ===
Loss: 1.6441, Accuracy: 0.3077

=== 모델링 기법 설명 ===
이 스크립트에서는 '월별 단어 빈도 벡터'를 1차원 시계열처럼 보고,
Conv1D(1D Convolutional Neural Network) 레이어를 통해 특징을 추출한 뒤
GlobalMaxPooling1D → Dense → Softmax 구조로 'down', 'neutr

In [7]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================
# 1) 파일 경로 지정
# ============================
# WORD_FILE  = './community/monthly_word_alt.csv'
CHART_FILE = '../chart/BINANCE_BTCUSDT_daily_UTC.csv'

# ============================
# 2) 월별 단어 빈도 데이터 로드
# ============================
df_words = pd.read_csv(
    WORD_FILE,
    sep=',',
    encoding='utf-8-sig'
)
# 컬럼: ['month','count','rankw','word','frequency']
# 'month'는 'YYYY-MM' 형식이라고 가정

# ============================
# 3) BTC 차트 데이터 로드 → 월별 등락률 계산
# ============================
df_chart = pd.read_csv(CHART_FILE, parse_dates=['date'])
df_chart['year_month'] = df_chart['date'].dt.to_period('M')

monthly_prices = (
    df_chart
    .groupby('year_month')['close']
    .agg(first='first', last='last')
    .reset_index()
)
monthly_prices['pct_change'] = (
    monthly_prices['last'] - monthly_prices['first']
) / monthly_prices['first']
monthly_prices['year_month'] = monthly_prices['year_month'].dt.strftime('%Y-%m')
monthly_prices['category'] = pd.cut(
    monthly_prices['pct_change'],
    bins=[-np.inf, -0.10, 0.10, np.inf],
    labels=['down', 'neutral', 'up']
)

# ============================
# 4) 월별 상위 10개 단어 추출
# ============================
df_words['year_month'] = df_words['month']
top10_per_month = (
    df_words
    .sort_values(['month','frequency'], ascending=[True, False])
    .groupby('month')
    .head(10)
    .reset_index(drop=True)
)
months_sorted = sorted(df_words['month'].unique())

# ============================
# 5) CNN 입력 데이터 준비 (월별 상위 10개 단어 빈도)
# ============================
X_list = []
for m in months_sorted:
    subset = top10_per_month[top10_per_month['month'] == m]
    freqs = subset['frequency'].tolist()
    if len(freqs) < 10:
        freqs += [0] * (10 - len(freqs))
    X_list.append(freqs)
X_arr = np.array(X_list, dtype=float)
X = X_arr.reshape((X_arr.shape[0], X_arr.shape[1], 1))

# y: 카테고리 인코딩
le = LabelEncoder()
y = le.fit_transform(monthly_prices.set_index('year_month').loc[months_sorted, 'category'].values)

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# 6) 1D CNN 모델 정의 및 학습
# ============================
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(10,1)),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=8,
    validation_split=0.1,
    callbacks=[es],
    verbose=2
)

# ============================
# 7) 모델 평가 및 예측
# ============================
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"==== 테스트 세트 평가: Loss={loss:.4f}, Accuracy={accuracy:.4f} ====\n")

probabilities = model.predict(X, verbose=0)
up_prob = probabilities[:, 2]  # “up” 클래스 확률
pred_classes = np.argmax(probabilities, axis=1)
predicted_labels = le.inverse_transform(pred_classes)

# ============================
# 8) 결과 정리
# ============================
results_df = pd.DataFrame({
    'month':             months_sorted,
    'actual_category':   monthly_prices.set_index('year_month').loc[months_sorted, 'category'].values,
    'predicted_category': predicted_labels,
    'up_probability':    up_prob.round(4),
    'pct_change':        monthly_prices.set_index('year_month').loc[months_sorted, 'pct_change'].values
})

# ============================
# 9) 연도·월 분리
# ============================
results_df[['year', 'month_num']] = results_df['month'].str.split('-', expand=True)
results_df['year'] = results_df['year'].astype(int)
results_df['month_num'] = results_df['month_num'].astype(int)

# ============================
# 10) 가로형 피벗: 각 필드를 연도별 1~12월 컬럼으로
# ============================

# ============================
# 10-1) actual_price 피벗: 월별 종가(‘last’) 기준으로
# ============================
# 10-1-a) monthly_prices에서 'year', 'month_num', 'last' 컬럼 준비
monthly_prices[['year', 'month_num']] = monthly_prices['year_month'].str.split('-', expand=True)
monthly_prices['year'] = monthly_prices['year'].astype(int)
monthly_prices['month_num'] = monthly_prices['month_num'].astype(int)

# 10-1-b) 피벗 생성: index=year, columns=month_num, values=last (월말 종가)
pivot_actual = (
    monthly_prices
    .pivot(index='year', columns='month_num', values='last')
)
pivot_actual.columns = [f"{m:02d}" for m in pivot_actual.columns]
pivot_actual = pivot_actual.reindex(columns=[f"{m:02d}" for m in range(1, 13)])

# 10-2) predicted_category 피벗
pivot_pred = (
    results_df
    .pivot(index='year', columns='month_num', values='predicted_category')
)
pivot_pred.columns = [f"{m:02d}" for m in pivot_pred.columns]
pivot_pred = pivot_pred.reindex(columns=[f"{m:02d}" for m in range(1, 13)])

# 10-3) up_probability 피벗
pivot_prob = (
    results_df
    .pivot(index='year', columns='month_num', values='up_probability')
)
pivot_prob.columns = [f"{m:02d}" for m in pivot_prob.columns]
pivot_prob = pivot_prob.reindex(columns=[f"{m:02d}" for m in range(1, 13)])

# ============================
# 11) CSV로 저장
# ============================
pivot_actual.to_csv('./actual_category_by_year.csv', encoding='utf-8-sig')
pivot_pred.to_csv('./predicted_category_by_year.csv', encoding='utf-8-sig')
pivot_prob.to_csv('./up_probability_by_year.csv', encoding='utf-8-sig')

# ============================
# 12) 출력 확인
# ============================
print("==== 연도별 월별 실제 카테고리 ====")
print(pivot_actual)
print("\n==== 연도별 월별 예측 카테고리 ====")
print(pivot_pred)
print("\n==== 연도별 월별 up_probability ====")
print(pivot_prob)

Epoch 1/20


c:\Users\kmj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 - 1s - 217ms/step - accuracy: 0.3111 - loss: 6.5404 - val_accuracy: 0.1667 - val_loss: 11.5650
Epoch 2/20
6/6 - 0s - 26ms/step - accuracy: 0.2889 - loss: 3.0049 - val_accuracy: 0.3333 - val_loss: 1.4696
Epoch 3/20
6/6 - 0s - 26ms/step - accuracy: 0.3333 - loss: 2.5033 - val_accuracy: 0.5000 - val_loss: 1.3510
Epoch 4/20
6/6 - 0s - 20ms/step - accuracy: 0.3333 - loss: 1.4908 - val_accuracy: 0.1667 - val_loss: 3.7558
Epoch 5/20
6/6 - 0s - 21ms/step - accuracy: 0.3778 - loss: 1.9378 - val_accuracy: 0.0000e+00 - val_loss: 3.5757
Epoch 6/20
6/6 - 0s - 20ms/step - accuracy: 0.3111 - loss: 1.5893 - val_accuracy: 0.1667 - val_loss: 1.7611
==== 테스트 세트 평가: Loss=3.0187, Accuracy=0.3846 ====

==== 연도별 월별 실제 카테고리 ====
            01         02        03        04        05        06        07  \
year                                                                          
2020   7200.85    9384.61   8531.88   6642.92   8826.96  10200.77   9232.00   
2021  29331.69   33526.37  49587.03  58720.4

# 저장 하기

Epoch 1/20


c:\Users\kmj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 - 2s - 297ms/step - accuracy: 0.2667 - loss: 5.5580 - val_accuracy: 0.1667 - val_loss: 9.7527
Epoch 2/20
6/6 - 0s - 22ms/step - accuracy: 0.4000 - loss: 4.3630 - val_accuracy: 0.1667 - val_loss: 5.7708
Epoch 3/20
6/6 - 0s - 34ms/step - accuracy: 0.2444 - loss: 2.0718 - val_accuracy: 0.3333 - val_loss: 1.7476
Epoch 4/20
6/6 - 0s - 23ms/step - accuracy: 0.2667 - loss: 2.0483 - val_accuracy: 0.6667 - val_loss: 1.0911
Epoch 5/20
6/6 - 0s - 25ms/step - accuracy: 0.3556 - loss: 2.1844 - val_accuracy: 0.1667 - val_loss: 2.9816
Epoch 6/20
6/6 - 0s - 37ms/step - accuracy: 0.2889 - loss: 1.7096 - val_accuracy: 0.5000 - val_loss: 0.9310
Epoch 7/20
6/6 - 0s - 24ms/step - accuracy: 0.2444 - loss: 1.7274 - val_accuracy: 0.5000 - val_loss: 1.3523
Epoch 8/20
6/6 - 0s - 21ms/step - accuracy: 0.3778 - loss: 1.6314 - val_accuracy: 0.1667 - val_loss: 2.6197
Epoch 9/20
6/6 - 0s - 33ms/step - accuracy: 0.4000 - loss: 1.2401 - val_accuracy: 0.6667 - val_loss: 0.9810
==== 테스트 세트 평가: Loss=1.5764, Accuracy=